# Vehicle Detection Project

The goals / steps of this project are the following:

* Define functions to compute input features from dataset car/non-car images.
    - Color space change, Spatial binning, Color histogram, HOG with subsampling
* Combine (concatenate) input features, normalize/scale (using sklearn StandardScaler) them.
* Using a balanced dataset and shuffled train/test splits, train a single or ensemble of classifiers to detect cars.
    - Linear SVM, Decision trees, Deep neural networks
* Implement a generic sliding window technique that feeds patches from a given image to classifier for classification.
    - Use multiple window sizes
    - Come up with efficient scan strategy (e.g. only bottom half of image)
* Implement heatmap thresholding to combine multiple detections and remove false positives
* Use information from multiple frames to remove spurious false positives that appear in one or small number of frames.
    - Calculate motion trajectory if required


In [ ]:
import numpy as np
import cv2
import glob
import os
import random
import pickle
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
### Helper Functions

def readImage(imfile):
    img = cv2.imread(imfile)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def plotImageFiles(filenames):
    for f in filenames:
        fig = plt.figure()
        #fig.set_size_inches(3,6)
        img = readImage(f)
        plt.title(f)
        plt.axis('off')
        plt.imshow(img)
        
def plotImage(img, title=None, cmap=None):
    fig = plt.figure()
    #fig.set_size_inches(4,8)
    if title is not None:
        plt.title(title)
    plt.axis('off')
    if cmap is None:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap=cmap)
    
def plotMultipleImages(images, labels=None, ptitle=None, cmap=None):
    """ This function will plot the images specified in a
    single plot.
    """
    numImages = len(images)
    ii = 1
    for img in images:
        fig = plt.figure()
        if labels is not None:
            plt.title(labels[ii-1], fontsize="xx-small")
        plt.axis('off')
        if cmap is not None:
            plt.imshow(img.squeeze(), cmap=cmap)
        else:
            plt.imshow(img.squeeze())
        ii += 1
    
    
print("*** Helper Functions Defined ***")

### Compute Input Features

In [ ]:
def getBinSpatialFeatures(img, size=(32, 32)):
    """ Computes spatially binned color features
    """
    features = cv2.resize(img, size).ravel() 
    return features

def getColorHistogramFeatures(img, nbins=32, bins_range=(0, 256)):
    """ Computes color histogram of features for each image channel
    and combines them into a single feature vector.
    """
    channelhists = []
    for i in range(0, img.shape[2]):
        channelhist = np.histogram(img[:,:,i], bins=nbins, range=bins_range)
        channelhists.append(channelhist)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate(channelhists)
    return hist_features

def getHOGFeatures(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    """ Computes and returns HOG features and visualization (optional).
    """
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features
    
def extractFeatures(img, color_space='RGB', spatial_size=(32, 32),
                    hist_bins=32, orient=9,
                    pix_per_cell=8, cell_per_block=2, hog_channel=0,
                    spatial_feat=True, hist_feat=True, hog_feat=True):    
    """ Returns a combined image feature vector for given image.
    """
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = getBinSpatialFeatures(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = getColorHistogramFeatures(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(getHOGFeatures(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = getHOGFeatures(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

def batchExtractFeatures(imgfiles, color_space='RGB', spatial_size=(32, 32),
                         hist_bins=32, orient=9, 
                         pix_per_cell=8, cell_per_block=2, hog_channel=0,
                         spatial_feat=True, hist_feat=True, hog_feat=True):
    batch_features = []
    for imfile in imgfiles:
        img = readImage(imfile)
        features = extractFeatures(img, color_space, spatial_size, hist_bins, orient,
                                   pix_per_cell, cell_per_block, hog_channel,
                                   spatial_feat, hist_feat, hog_feat)
        batch_features.append(features)
    return batch_features

def plotSampleHogImages():
    carfiles = glob.glob('data/vehicles/**/*.png', recursive=True)
    noncarfiles = glob.glob('data/non-vehicles/**/*.png', recursive=True)
    samples = min(len(carfiles), len(noncarfiles))
    ind = random.randint(0, samples-1)
    carimg = readImage(carfiles[ind])
    noncarimg = readImage(noncarfiles[ind])
    gscarimg = cv2.cvtColor(carimg, cv2.COLOR_RGB2GRAY)
    gsnoncarimg = cv2.cvtColor(noncarimg, cv2.COLOR_RGB2GRAY)
    _, carhogvis = getHOGFeatures(gscarimg, orient=9, pix_per_cell=4, cell_per_block=2, vis=True, feature_vec=True)
    _, noncarhogvis = getHOGFeatures(gsnoncarimg, orient=9, pix_per_cell=4, cell_per_block=2, vis=True, feature_vec=True)
    plotMultipleImages([carimg, carhogvis], cmap="gray")

print("*** Input Features Computation Functions Defined ***")

### Classifier Training

In [ ]:
def getClassifierParams():
    params = dict()
    params['color_space'] = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    params['orient'] = 9  # HOG orientations
    params['pix_per_cell'] = 8 # HOG pixels per cell
    params['cell_per_block'] = 2 # HOG cells per block
    params['hog_channel'] = "ALL" # Can be 0, 1, 2, or "ALL"
    params['spatial_size'] = (16, 16) # Spatial binning dimensions
    params['hist_bins'] = 16    # Number of histogram bins
    params['spatial_feat'] = False # Spatial features on or off
    params['hist_feat'] = False # Histogram features on or off
    params['hog_feat'] = True # HOG features on or off
    return params
    

def getClassifier(datafile = "./classifier/svc.pickle", forcetrain=False):
    """ Loads a pre-trained classifier if available or trains a new one.
    It takes care of loading up the dataset images, extracting the image
    features
    """
    if os.path.isfile(datafile) and not forcetrain:
        print("--> Returning saved classifier")
        with open(datafile, "rb") as f:
            pickledata = pickle.load(f)
            clf = pickledata["classifier"]
            return clf
    carfiles = glob.glob('data/vehicles/**/*.png', recursive=True)
    noncarfiles = glob.glob('data/non-vehicles/**/*.png', recursive=True)
    numsamples = min(len(carfiles), len(noncarfiles))
    params = getClassifierParams()
    print("--> Extracting features. Numsamples = ", numsamples)
    carfeatures = batchExtractFeatures(carfiles[0:numsamples], color_space=params['color_space'],
                        spatial_size=params['spatial_size'], hist_bins=params['hist_bins'], 
                        orient=params['orient'], pix_per_cell=params['pix_per_cell'],
                        cell_per_block=params['cell_per_block'], 
                        hog_channel=params['hog_channel'], spatial_feat=params['spatial_feat'], 
                        hist_feat=params['hist_feat'], hog_feat=params['hog_feat'])
    noncarfeatures = batchExtractFeatures(noncarfiles[0:numsamples], color_space=params['color_space'],
                        spatial_size=params['spatial_size'], hist_bins=params['hist_bins'], 
                        orient=params['orient'], pix_per_cell=params['pix_per_cell'],
                        cell_per_block=params['cell_per_block'], 
                        hog_channel=params['hog_channel'], spatial_feat=params['spatial_feat'], 
                        hist_feat=params['hist_feat'], hog_feat=params['hog_feat'])
    X = np.vstack((carfeatures, noncarfeatures)).astype(np.float64)                        
    print("--> Scaling features")
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    # Define the labels vector
    y = np.hstack((np.ones(len(carfeatures)), np.zeros(len(noncarfeatures))))
    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)
    # Use a linear SVC 
    svc = LinearSVC()
    # Check the training time for the SVC
    print("--> Starting to Train Classifier")
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print("--> ", round(t2-t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
    print('--> Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    print("--> Saving Classifier")    
    t=time.time()
    with open(datafile, "wb") as f:
        pickle.dump( { 'classifier': svc,}, f, pickle.HIGHEST_PROTOCOL )
    return svc

svc = getClassifier()

print("*** Classifier Training Functions Defined ***")

### Sliding Window Technique

In [ ]:
    
print("*** Sliding Window Technique Functions Defined ***")


### Combining Overlaps and False Positives Removal

In [ ]:
print("*** Overlap Combining and False Positives Removal Functions Defined ***")


### Overall Pipeline

In [ ]:
def pipeline():
    pass

In [ ]:
def testPipelineOnTestImages():
    tfiles = glob.glob('test_images/test*.jpg')
    for tfile in tfiles:
        timg = mpimg.imread(tfile)
        plt.figure(figsize=(10, 12))
        plt.imshow(timg)

#testPipelineOnTestImages()


### Vehicle Detection on Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#TODO: Basic pre-processing

def process_image(image):
    return image

In [ ]:
prevllfit = None
prevrlfit = None
outvideofile = 'project_video_output.mp4'
pvideo = VideoFileClip("project_video.mp4")
ovideo = pvideo.fl_image(process_image)
%time ovideo.write_videofile(outvideofile, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(outvideofile))